In [2]:
# Auto-reload setup
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

from data_manipulation_func import load_epex_data, clean_data
print('Successful import')
df = load_epex_data('../data/2021_prices_raw.csv')
df = clean_data(df)
df.to_csv('../data/clean/2021_prices_clean.csv', index=False)

Successful import


In [10]:
from data_manipulation_func import exploitable_data

folderpath = '../data/clean'
df_concat = exploitable_data(folderpath)
df_concat.to_csv(f'{folderpath}/exploitable/multiple_years.csv', index=False)